# LLM for alle - Introduksjonskurs til språkmodeller med Python og Azure OpenAI

# Introduksjon

Etter prosjektet vi gjorde for Elkem fikk vi hands-on erfaring med å utvikle og bruke språkmodeller (LLMer) i praksis. Vi så hvor effektivt dette kan være for å spare tid, redusere kostnader og automatisere manuelle oppgaver på en måte som enkelt kan gjenbrukes i andre prosjekter.

I dette kurset bruker vi et konkret og gjenkjennbart case: analyse av resultatene fra en medarbeiderundersøkelse – en oppgave mange sliter med å strukturere. Vi skal lære dere å koble dere til en LLM via API, bruke LangChain Expression Language (LCEL) til å bygge AI-kjeder og hente ut strukturert innsikt med Pydantic – alt i én oversiktlig notebook.





# Oppgave
Du jobber i et IT-selskap og har fått i oppgave å analysere svarene fra en intern medarbeiderundersøkelse. Undersøkelsen er anonym, og du har fått tilsendt en CSV-fil med 50 tilbakemeldinger – én per ansatt. Målet er å finne ut hva folk er fornøyde eller misfornøyde med, og særlig se nærmere på temaene Nettverk, Opplæring og IT-support, som ledelsen er ekstra interessert i. Tilbakemeldinger som ikke passer i disse kategoriene skal også få sin plass. Til slutt skal du lage en oppsummering som kan sendes til ledelsen.

For å jobbe effektivt bruker du en språkmodell til å hjelpe deg med både kategorisering og oppsummering. 

**Oppgaven blir dermed å bruke en språkmodell til å kategorisere samt oppsummere tilbakemeldingene fra undersøkelsen.** 


## Datasett

In [1]:
# Import the raw data containing the feedback. There is one row per employee, and all employees answered the questionnaire. 

import pandas as pd
pd.set_option('display.max_colwidth', None) # Ensure no truncated output of dataframe

enr_path = "../files/IT_survey.csv"
df = pd.read_csv(enr_path)
df.head()


,ID,Category,Feedback
0,1,Network,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity."
1,2,Business needs,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches."
2,3,Security,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated."
3,4,Quality of tools,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity."
4,5,IT-support,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty."


Før vi går løs på oppgaven skal du bli bedre kjent med verktøyene og metodene vi bruker. 

## Bruk av språkmodeller gjennom en API

API = Application Programming Interface. Et grensesnitt som lar forskjellige programmer kommunisere med hverandre. 

### En vanlig LLM-spørring
Enkle LLM-spørringer er bygget opp av noen sentrale deler:

1. Tilkobling til en ressurs, som feks. Azure OpenAI

2. En prompt, som vil si en tekstbasert forespørsel/instruks

3. Sending av prompt til språkmodellen for å hente en respons


Som en del av tilkoblingen er det vanlig å oppgi en temperaturparameter. Denne parametreren angir nivået av presisjon du ønsker å få i responsen fra språkmodellen, og kan enten måles på en skala fra "low" til "high" eller numerisk fra 0 til 1.
Hvis denne parameteren settes til "low"/nærme 0 tillater du liten grad av variasjon og kreativitet i responsen, og du vil få tryggere og mer forutsigbare svar. Hvis den derimot settes til høy/nærme 1 tillater du større grad av kreativitet og detaljer, men vil følgelig også få en mer uforutsigbar respons. 

In [6]:
# Connect through the API
from langchain_openai import AzureChatOpenAI
from dotenv import find_dotenv, load_dotenv
import os
from IPython.display import Markdown # Pretty output


# Get environment variables
load_dotenv(find_dotenv(), override=True)


llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_GPT_4O_MINI"),
    model=os.getenv("OPENAI_MODEL_GPT_4O-MINI", default="gpt-4o-mini"),
    temperature=0,
)

reasoning_llm = AzureChatOpenAI(
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_O4_MINI"),
    model=os.getenv("OPENAI_MODEL_O4-MINI", default="o4-mini"),
    reasoning_effort="medium",
)

In [7]:
# Generate the prompt
prompt = 'Hei!'

# Send the prompt and recieve a response
response = llm.invoke(prompt)

# Show the response from the model
response


AIMessage(content='Hei! Hvordan kan jeg hjelpe deg i dag?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 9, 'total_tokens': 20, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence':

In [8]:
# Show only the content of the response
response.content

'Hei! Hvordan kan jeg hjelpe deg i dag?'

In [ ]:
# Test out the temperature-parameter

# Define the prompt
prompt = 'Hei! Fortell meg en morsom historie om en katt. Skriv maks 5 setninger.'

# Set temperature (0 = most predictable, 1 = most creative and random)
temperature = 0.9  # Try to switch this out with 0.1 and 0.5 to see the difference. 

# Send the prompt and receive a response
response = llm.invoke(prompt, temperature=temperature)

# Print the response from the model
print(response)

content='Det var en gang en katt som het Maja, som trodde hun var en superhelt. Hver dag satte hun på seg et lite rødt teppe og sprang rundt i huset for å "redde" kosedyrene sine fra den fryktede støvsugeren. En dag, mens hun spratt for å redde en kattunge fra støvsugerens "klør", traff hun en haug med puter og landet som en ekte akrobat. Maja fikk en stor applaus fra familien, som trodde hun hadde planlagt hele stuntet. Fra den dagen av ble hun kjent som "Kattematadoren", og støvsugeren ble hennes største fiende!' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 147, 'prompt_tokens': 28, 'total_tokens': 175, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_7a53abb7a2', 'prompt_filter_results': [{'prompt_index': 0, 'content_

## Hvordan lage en enkel AI-kjede med LangChain Expression Language (LCEL)

LCEL er en metode for å bygge og kjøre såkalte *kjeder* i LangChain på. Kjeder, eller chains, brukes for å koble sammen ulike AI-komponenter. F.eks. kan språkmodeller, datakilder og logikk kobles sammen til én sammenhengende prosess, dvs. en kjede. LCEL gir et standardisert språk for å definere disse kjedene, og er brukervennlig fordi man slipper å lage alt manuelt med kode. Med andre ord får du en "oppskrift" på hvordan AI-komponentene dine skal jobbe sammen.

**Fordelene med LCEL**
1. Støtter parallell og asynkron kjøring - Ulike deler av kjeden kan kjøre samtidig, og systemet kan behandle flere forespørsler på en gang. Dermed kan oppgaver behandles raskere. 
2. Strømming av resultater - Man kan begynne å se svar mens AI-en fremdeles jobber. (Passer spesielt godt for chatbaserte løsninger)
3. Enkel feilsøking med LangSmith - Når kjedene blir komplekse er det viktig å kunne se hva som har blitt gjort underveis. LCEL logger automatisk alt til LangSmith, som gjør det enklere å feilsøke. 
4. Standardisert - Alle kjeder i LCEL bruker samme grensesnitt, som gjør dem enkle å kombinere og gjenbruke på tvers av prosjekter. 

LCEL bruker en pipe-operator (|) til å koble sammen ulike trinn i kjeden. Den tar ut data fra én komponent og sender den direkte som input til neste komponent. LCEL bruker også PromptTemplate, som kan tenkes på som en mal for teksten du sender til språkmodellen. PromptTemplate gjør det enkelt å lage dynamiske meldinger ved at man kan sette inn variabler i teksten, litt som en oppskrift der du fyller inn det som mangler før det sendes til AI-modellen. Fordelen med PromptTemplate er at man kan lage én mal, og bruke den med ulike data. Det hjelper deg også med å skille selve teksten fra logikken, og kan gjøre prosessen sikrere ved at man unngår feil som kan oppstå ved manuell string-manipulasjon. Vi skal nå se på noen eksempler med LCEL som bruker pipe-operator og PromptTemplate.

Kilde: xxx

In [9]:
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
    Hi! Please talk like a {role}.
    """
)
prompt

PromptTemplate(input_variables=['role'], input_types={}, partial_variables={}, template='\n    Hi! Please talk like a {role}.\n    ')

In [10]:
chain = prompt | llm

chain.invoke({"role": "pirate"})

AIMessage(content="Ahoy, matey! What be ye wantin' to chat about on this fine day upon the high seas? Arrr!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 28, 'prompt_tokens': 18, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_ded0d14823', 'prompt_filter_results': [{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}}}], 'finish_reason': 'stop', 'logprobs': None, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {

Som dere ser returnerer modellen mye mer enn kun svaret på prompten vår, nemlig en hel haug med meta-data. Dette bryr vi oss ikke så mye om i dette kurset, så vi bruker StrOutputParser for å kun få ut tekst-responsen fra modellen.

In [ ]:
from langchain_core.output_parsers import StrOutputParser
chain_str = chain | StrOutputParser()

chain_str.invoke({"role": "pirate"})


"Ahoy, matey! What be ye wantin' to chat about on this fine day upon the high seas? Arrr!"

#### Prøv selv!

In [ ]:
# PromptTemplate as an email-template. 

prompt = PromptTemplate.from_template(
    """
    Finn på en prompt med variabler som f.eks. informerer deg om hvor mange nye emails du har fått. Prøv deg frem, ingenting er feil!
    """
)  

chain = prompt | llm
chain_str = chain | StrOutputParser()

chain_str.invoke({"variabel_1":...}) # Fill out the blanks!

In [25]:
# FASIT!
# PromptTemplate as an email-template. 
from langchain_core.prompts import PromptTemplate
prompt = PromptTemplate.from_template(
    """
    Hi {name}, you have recieved {number} of new emails. 
    """
) # Feel free to change the variables or rewrite the prompt. 

chain = prompt | llm
chain_str = chain | StrOutputParser()

chain_str.invoke({"name": "Ola", "number": "3"}) # Fill out the blanks!


"Hi! It looks like you've received three new emails. Would you like help with anything specific regarding them?"

## Enkel kategorisering av hver enkelt tilbakemelding

### Bruk LLMen til kategorisering av tilbakemeldingene
I første omgang av kategoriseringen er vi interesserte i å se hvor mange av tilbakemeldingene som passer innenfor de kategoriene ledelsen ønsket et ekstra fokus på, nemlig Network, Training og IT-support. Vi kan be LLMen om å utføre denne kategoriseringen ved å gi den tilgang på feedback-dataen. Husk på de sentrale delene av enkle LLM-kall, og benytt deg av LCEL som vist i forrige eksempel. 

In [12]:
categorize_prompt = PromptTemplate.from_template(
"""
Categorize the following feedback into one of the following categories:
- Network
- Training
- IT-support
- Other

Feedback:
<feedback>
{feedback}
</feedback>
"""
)

categorize_chain = categorize_prompt | llm | StrOutputParser()

categorize_chain.invoke({"feedback": "I am very happy with the IT support I received last week."})

'Category: IT-support'

## Structured output

### Pydantic 
Pydantic er et Python-bibliotek for datavalidering og datastrukturering. Hovedklassen i Pydantic heter BaseModel og er klassen vi arver fra når vi lager våre egne datamodeller. Når vi arver fra BaseModel får vi automatisk funksjonalitet som kan:
1. Validere innhold du sender inn
    - Eks: Du definerer en liste med godkjente land, "Norge, Sverige, Finland", da vil ikke pydantic godkjenne "Australia".
2. Konvertere data til riktig type
    - Eks: Du definerer at output skal være en int og sender inn '1', pydantic vil da returnere 1 (som int)
3. Påtvinge JSON-formatering
    - Sørger for at responsen LLMen gir matcher JSON-skjemaet til Pydantic. Dette gjør at vi kan være sikre på strukturen til outputen, som for eksempel er svært nyttig om vi ønsker å bruke outputen fra en modell som input i en annen. 



### Structured output 
I denne konteksten refererer structured output til strategien og verktøyene vi bruker for å forsikre oss om at dataen vår blir organisert på en måte vi definerer på forhånd. 
Funksjonalitet i Pydantic biblioteket lar oss bestemme strukturen på outputen gjennom å spesifisere felter, definere typer (eks. int, str, List[int]) og validere data.
I dette kurset kommer vi til å bruke BaseModel til å lage våre egne klasser for å sikre at det vi mottar fra modellene når vi prompter de kommer på akkurat den formen vi ønsker. 

In [ ]:
from pydantic import BaseModel, Field

class Categorize(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category : str = Field(description="The best fitting category. Only one.") 


I eksempelet over har vi laget vår egen klasse 'Categorize' som arver av BaseModel.
For variabelen category har vi brukt type hinting ( x: type ) for å definere typen Pydantic skal forvente at category er lik. Dette for ekesempel blir da ulovlig

In [14]:
Categorize(category=1)

ValidationError: 1 validation error for Categorize
category
  Input should be a valid string [type=string_type, input_value=1, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/string_type

Mens dette er helt ok

In [15]:
Categorize(category="Network")

Categorize(category='Network')

Vi bruker også funksjonen Field, denne kan du bruke til å sette standardverdier, valideringsregler og beskrivelser.
I dette kurset bruker vi kun beskrivelse, men for de spesielt interesserte kan dere lese mer om funksjonaliteten [her](https://docs.pydantic.dev/latest/concepts/fields/).

For å få LLMen til å skjønne at den må følge reglene vi har definert i klassen vår bruker vi wrapperen with_structured_output(...). 
Denne wrapper kallet vårt til språkmodellen med logikk som forsikrer at outputen følger strukturen vi har definert i klassen.

In [26]:
categorize_chain_structured_output = categorize_prompt | llm.with_structured_output(
    Categorize,
    method="json_schema", # Demands JSON-schema for output
    strict=True           # The model must adhere to the specified schema, no extra fields or missing fields are allowed. All types must be an exact match. 
)

categorize_chain_structured_output.invoke(
    {"feedback": "I am very happy with the IT support I received last week."}
)

Categorize(category='IT-support')

Vi kan bruke Literal for å definere våre egne typer til å bruke for type hinting. Vi kan type hinte variabler med CATEGORIES under og dette vil da modellen tolke på samme måte som at en int bare har lov til å være et heltall har denne "typen" bare lov til å være en av verdiene listet opp i Literal-objektet. Med andre ord får ikke modellen returnert noen andre kategorier enn de som er definert i lista. 


In [27]:
from typing import Literal

CATEGORIES = Literal[
    "Network",
    "Training",
    "IT-support",
    'Other'
]

La oss sette sammen det vi har lært. Legg merke til at vi ikke lenger lister kategoriene i prompten (som på ingen måte garanterer at vi kun får ut en av kategorierne vi øsnker), men definerer de som et krav til strukturen på outputen fra modellen. 

In [30]:
from typing import Literal

categorize_prompt = PromptTemplate.from_template(
"""
Categorize the following feedback into the provided categories.

Feedback:
<feedback>
{feedback}
</feedback>
"""
)



class CategorizeFromOptions(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category: CATEGORIES = Field(
        description="Chosen category for the feedback. Choose 'Other' if the other categories provided are not a good fit."  
    )


categorize_chain_structured_output = categorize_prompt | llm.with_structured_output(
    CategorizeFromOptions,
    method="json_schema",
    strict=True,
)

In [31]:
result = categorize_chain_structured_output.invoke(
    {"feedback": "I am very happy with the IT support I received last week."}
)

result


CategorizeFromOptions(category='IT-support')

## Chain-of-thought

Chain of Thought (CoT) er en teknikk innen prompt engineering som hjelper språkmodeller med å løse oppgaver som krever flere tankesteg. I stedet for å hoppe rett til svaret, blir modellen ledet gjennom en logisk og trinnvis prosess som kan gi mer presise og gjennomtenkte svar [1]. 

Du kan altså be modellen om å "tenke høyt" under oppgaven og forklare stegene sine før den leverer et endelig svar. Dette ber du om i prompten som sendes inn. 

Eksempel på en prompt **uten** CoT: 

    Prompt: "Hvor mange armer har Eline og Kaspara?"

    Svar: "4"

Eksempel på en prompt **med** CoT:

    Prompt: "Hvor mange armer har ELine og Kaspara? Tenk trinn for trinn."

    Svar: "En person har to armer. To personer betyr 2x2 = 4 armer. Svaret er 4."

Det kan være fordelaktig å bruke CoT når man jobber med komplekse oppgaver, da nøyaktigheten på outputet fra modellen øker når den "får lov" til å jobbe seg gjennom problemet. 

I tillegg kan du se hvordan modellen tenker, som gjør det lettere for deg å evaluere svaret. Det blir også lettere å se hvor det gikk galt hvis modellen svarer feil.

##### *Kilder*
[1] xxxx, Link: https://www.ibm.com/think/topics/chain-of-thoughts 

In [33]:
class CategorizeCot(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    chain_of_thought: str = Field(
        description="Use this space to think through the categorization."  # We have defined a variable that explicitly asks the model to think in chain-of-thought
    )
    category: CATEGORIES = Field(
        description="Chosen category for the feedback. Choose 'Other' if the other categories provided are not a good fit."  # prompt
    )


categorize_chain_cot = categorize_prompt | llm.with_structured_output(
    CategorizeCot,
    method="json_schema",
    strict=True,
)

In [39]:
result = categorize_chain_cot.invoke("The internet-speed is so slow I could chase after the IT-survice guy with a baseball-bat!")
print(result.chain_of_thought) 
print(result.category)

The feedback specifically mentions issues with internet speed, which falls under the category of 'Network'. The frustration expressed indicates a problem with the network performance rather than IT support or training.
Network


# Resonneringsmodell

Resonneringsmodeller, som Azure Open AI sin O4-mini-modell, er språkmodeller som er spesielt trent på å tenke før de svarer. Altså *resonnerer* de seg frem til et svar, i motsetning til en typisk LLM som gir raske svar. 
Resonneringsmodeller bruker chain of thought til å bryte ned oppgaven i mindre deler, for å så jobbe seg gjennom problemstillingen stegvis [3]. Dette er fordelaktig å bruke til oppgaver som krever kompleks problemløsning, logisk tenkning som koding eller matematikk eller til oppgaver med flere steg. Det vil også være fordelaktig å bruke i situasjoner der nøyaktighet og forklarbarhet er viktig [2]. 


##### Kilder
[2] OpenAI, Link: https://platform.openai.com/docs/guides/reasoning?api-mode=chat
[3] Tzur, A. *(2025)*. Resonneringsmodeller: AI som tenker før de svarer. AIavisen: https://aiavisen.no/resonneringsmodeller/ 

### Eksempel på resonnering vs. ikke resonnering

In [57]:
# Example

prompt = PromptTemplate.from_template("""
A rope ladder hangs over the side of a boat. The rungs are 30 cm apart. The tide rises 1 meter every hour. After 3 hours, how many rungs will be underwater?
""")

reasoning_chain = prompt | reasoning_llm | StrOutputParser()

print(reasoning_chain.invoke({}))

none_reasoning_chain = prompt | llm | StrOutputParser()

print(none_reasoning_chain.invoke({}))

Because the ladder (and boat) float up with the tide, the water‐line on the ladder stays exactly where it was.  So however many rungs were underwater at the start will still be underwater after 3 hours.  (If none were underwater to begin with, none will be after 3 hours.)
If the tide rises 1 meter every hour, after 3 hours, the tide will have risen:

\[
1 \text{ meter/hour} \times 3 \text{ hours} = 3 \text{ meters}
\]

Since the rungs of the ladder are 30 cm apart, we need to convert the rise in tide from meters to centimeters:

\[
3 \text{ meters} = 3 \times 100 \text{ cm} = 300 \text{ cm}
\]

Now, to find out how many rungs will be underwater, we divide the total rise in tide by the distance between the rungs:

\[
\frac{300 \text{ cm}}{30 \text{ cm/rung}} = 10 \text{ rungs}
\]

Therefore, after 3 hours, **10 rungs** will be underwater.


Aha! Her ser vi et eksempel på at resonneringsmodellen tenker riktig fordi den bryter ned problemet, mens det vanlige llm-kallet uten resonnering går fem på lureoppgaven og svarer feil. 

# Nå lar vi modellen gå løs på hele datasettet

Nå skal vi bruke verktøyene vi har lært til å utføre den første oppgaven, nemlig å kategorisere tilbakemeldingene på tvers av datasettet. 

In [59]:
def categorize_single_feedback(feedback: str) -> str:       # Function to invoke model
    result = categorize_chain_structured_output.invoke(
        {"feedback": feedback}
    )                                       
    return result.category

df["AI Classification"] = df["Feedback"].apply(
    lambda feedback: categorize_single_feedback(feedback)   # Applies function to all rows in dataset
)

In [40]:
df 

,ID,Category,Feedback,AI Classification
0,1,Network,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity.",Network
1,2,Business needs,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches.",Other
2,3,Security,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated.",Other
3,4,Quality of tools,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity.",Other
4,5,IT-support,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty.",IT-support
5,6,Quality of tools,"Our advanced software tools offer a mix of efficiency and creativity, but the occasional downtime introduces a fleeting sense of dismay.",Network
6,7,Security,"I feel more secure with the recent network defense upgrades, yet the complexity of the security software occasionally overwhelms my routine.",Network
7,8,Security,"The new cybersecurity measures give me confidence in our data protection, though frequently changing protocols sometimes cause confusion.",Other
8,9,Security,"Our enhanced firewall settings significantly lower threats, though frequent reminders to update can feel intrusive.",Other
9,10,IT-support,"The assistance from IT-support is generally effective, though sporadic technical mishaps occasionally undermine steady progress.",IT-support


Nå vil vi se nærmere på radene som falt under 'Other' og ikke ble kategorisert i første omgang. Til dette vil vi bruke en resonneringsmodell. 

In [60]:
class Categorize_others(BaseModel):
    "Categorization of a single feedback entry from an IT survey."
    category_take1 : str = Field(
        description="The best fitting category. Only one."
    )
    category_take2 : str = Field(
        description = "Do another take of the categorization. If the category is very similar to any of the predetermined categories 'Network','Training', 'IT-support' then rather use one of those"
    )
 
 
categorize_prompt_other = PromptTemplate.from_template(
"""
Categorize the following feedback from an IT-survey into the category that best describes the feedback.
 
Feedback:
<feedback>
{feedback}
</feedback>
"""
)
 
categorize_chain_structured_output_others = categorize_prompt_other | reasoning_llm.with_structured_output(
    Categorize_others,
    method="json_schema",
    strict=True,
)
 
# Retrieve indexes for all rows where AI classification = "Other"
other_indices = df[df['AI Classification'] == "Other"].index
 
 
result_batch = categorize_chain_structured_output_others.batch(
    [
    {"feedback": feedback} for feedback in df.loc[other_indices, "Feedback"]
    ]
)
 
df.loc[other_indices, "AI Classification"] = [result.category_take2 for result in result_batch]

In [61]:
df

# Per nå bruker den kategoriene vi fikk fra LLMen. Når den kategorieserer Other får vi mange nye og litt varierende kategorier. Det er også mange av de. Skulle vi bare brukt den som eksempel også bruke den ekte kategoriseringe her?

,ID,Category,Feedback,AI Classification
0,1,Network,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity.",Network
1,2,Business needs,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches.",Legacy Systems
2,3,Security,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated.",Authentication
3,4,Quality of tools,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity.",Maintenance
4,5,IT-support,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty.",IT-support
5,6,Quality of tools,"Our advanced software tools offer a mix of efficiency and creativity, but the occasional downtime introduces a fleeting sense of dismay.",Application performance
6,7,Security,"I feel more secure with the recent network defense upgrades, yet the complexity of the security software occasionally overwhelms my routine.",Network
7,8,Security,"The new cybersecurity measures give me confidence in our data protection, though frequently changing protocols sometimes cause confusion.",Security
8,9,Security,"Our enhanced firewall settings significantly lower threats, though frequent reminders to update can feel intrusive.",Network
9,10,IT-support,"The assistance from IT-support is generally effective, though sporadic technical mishaps occasionally undermine steady progress.",IT-support


Oi! Her fikk vi mange andre kategorier. Du kan sammenligne AI sin kategorisering med fasiten i kolonnen "Category" - reflekter litt rundt hvorvidt AI er for spesifik. Her kan f.eks. prompten skrives om for å få mer / mindre spesifikke kategorier. Kunne vi definert en til kategori som "eksempel" for modellen for å unngå så mange kategorier med små variasjoner.

In [ ]:
# Store the results from the categorization of "Others" in a separate variable that is not part of the actual categories.
# This can then be used in the tasks below without confusing the LLM about which categories it should recognize.

categorized_survey = df[['ID','Feedback','AI Classification']]
categorized_survey

,ID,Feedback,AI Classification
0,1,"The network infrastructure is robust and usually efficient, though intermittent glitches sometimes hamper productivity and clarity.",Network
1,2,"Our current infrastructure aligns with business needs, yet occasional rigidity in legacy systems hinders innovative approaches.",Infrastructure and Systems
2,3,"I appreciate the proactive approach to security, though the multi-step authentication process sometimes seems overly complicated.",Security
3,4,"Our tools maintain high quality with intuitive design and regular updates, even though rare performance lags can hinder productivity.",Tool Quality and Performance
4,5,"The IT-support team is consistently prompt and courteous, though occasional miscommunications lead to fleeting moments of uncertainty.",IT-support
5,6,"Our advanced software tools offer a mix of efficiency and creativity, but the occasional downtime introduces a fleeting sense of dismay.",Network
6,7,"I feel more secure with the recent network defense upgrades, yet the complexity of the security software occasionally overwhelms my routine.",Network
7,8,"The new cybersecurity measures give me confidence in our data protection, though frequently changing protocols sometimes cause confusion.",Cybersecurity
8,9,"Our enhanced firewall settings significantly lower threats, though frequent reminders to update can feel intrusive.",Security
9,10,"The assistance from IT-support is generally effective, though sporadic technical mishaps occasionally undermine steady progress.",IT-support


Nå har vi definert passende kategorier for alle tilbakemeldingene. 
Videre vil vi få modellen til å oppsummere per kategori, slik at vi sitter igjen med en overordnet oversikt. 

## Din tur!

Nå som kategoriseringen er gjennomført, og vi har forsikret oss om at alle radene har fått en passende kategori, kan vi gå videre til neste oppgave. Her er målet å samle informasjonen fra kategoriseringen til en overordnet oversikt. Lag en LCEL-kjede som tar resultatet fra forrige oppgave og oppsummerer per kategori ved hjelp av en LLM. Inkluder structured output. 

In [ ]:
# Tip 1: Create a good prompt! Use this to reflect on what the end goal is.
prompt = PromptTemplate.from_template("""
Well, how should I phrase this? Shouldn't there be some data included here as well?
""")

# Tip 2: Can you create a class that inherits from BaseModel to make this easier?
class SummarizeFeedback(BaseModel):
    "Description..."
    summary: str = ...

# Tip 3: Time to create the chain
summary_chain_structured_output = ...

# Tip 4: Call the model with the dataset from the survey
summary = ...

In [47]:

# Fasit
# 1. Prompt Template
summary_prompt = PromptTemplate.from_template("""
You are a domain expert in internal IT operations and organizational analysis. You will be provided with a dataset containing qualitative feedback from employees in an IT company. 
Each row in the dataset represents a feedback entry and is associated with a specific category.

For each category, carefully:
1. Read and interpret the feedback entries assigned to that category.
2. Identify core themes, recurring patterns, and contrasting opinions within that category.
3. Evaluate the feedback logically: What are the likely underlying causes of recurring issues or praises? Are there signs of systemic problems, isolated incidents, or misaligned expectations?
4. Summarize each category in 3 to 6 bullet points, highlighting key sentiments (positive and negative), representative concerns or compliments, and any significant outliers

Present your findings in a clean, professional way with one section per category. 

This is the employee feedback data: {survey_results}
""")

# 2. Class for structured output
class SummarizeFeedback(BaseModel):
    "Summary of the different categorizes recognized in the feedback from an IT-survey."
    summary : str = Field(
        description="For each category: Category name and 3-6 bullet points summarizing the category."
    )

# 3. Summary-chain
summary_chain_structured_output = summary_prompt | llm.with_structured_output(
    SummarizeFeedback,
    method="json_schema",
    strict=True,
)

# 4. Kall modellen med det kategoriserte datasettet fra undersøkelsen 
summary = summary_chain_structured_output.invoke({"survey_results": categorized_survey})



In [49]:
# A nice way to show the output from the model
from IPython.display import Markdown

display(Markdown(summary.summary))

### Network
- Generally robust and efficient, but intermittent glitches can hamper productivity.
- Occasional slow transmissions during heavy usage create instability.
- Network performance is satisfactory, yet unexpected downtime during peak hours raises concerns.

### Infrastructure and Systems
- Current infrastructure aligns with business needs, but legacy systems can hinder innovation.
- Outdated processes clash with modern expectations, causing operational bottlenecks.

### Security
- Proactive security measures instill confidence, though complexity can overwhelm users.
- Frequent alerts and reminders can feel intrusive, leading to slight irritation.
- Robust protocols foster a safe environment, but sporadic warnings create heightened alertness.

### Tool Quality and Performance
- Tools are generally high quality with intuitive design, but occasional performance lags disrupt productivity.
- Some tools lag behind expectations, causing frustration during critical operations.

### IT-support
- IT-support is prompt and courteous, but miscommunications can lead to uncertainty.
- Generally effective assistance, though sporadic delays in follow-up communications can cloud the experience.
- Customer service is engaging, but occasional oversights leave tasks unfinished.

### Training
- Training materials are well-presented, but rapid changes can create disconnects.
- Sessions cover a wide range of topics, but fast pacing sometimes results in uneven clarity.
- Hands-on exercises are engaging, yet the fast pace can be challenging for some employees.

### Project Management
- Efforts to merge IT and business strategies are recognized, but mismatches can lead to project delays.

### User Experience
- Digital tools support daily work, but interface inconsistencies can lead to minor critiques.

### System Performance
- Systems generally support business functions effectively, but delays in adapting to new trends create bottlenecks.

### Business Strategy
- IT solutions are aligned with business needs, but occasional oversights in market adaptability create frustrations.

# Rapport til ledelsen

I siste oppgave skal vi bruke oppsummeringen vi nettopp lagde til å generere en rapport som kan sendes til ledelsen. Rapporten skal gi mottakerne en oversikt over de viktigste punktene i medarbeideundersøkelsen og komme med noen forslag til endringer som kan skape forbedringer til neste år. 


In [51]:
# Oppgave 
report_prompt = PromptTemplate.from_template("""
Bla bla bla
""")

# 2. Class for structured output
class ReportForLeadership(BaseModel):
    "Raport for the leadership of an IT-company on results of an internal IT-survey."
    snappy_title : str = Field(
        description="A fitting title for the report. Must begin with '# ' to ensure easy markdown formatting."
    )
    key_takeaway:str = ...

# 3. Report-chain
report_chain_structured_output = ...

# 4. Kall modellen med oppsummeringen av kategoriene
report = ...



In [ ]:
# Fasit
report_prompt = PromptTemplate.from_template("""
You are an expert HR and technical operations analyst. I will provide you with a dataset of employee feedback collected from an IT company.

Your task is to deeply analyze this feedback and generate a concise executive-level summary report in markdown format that includes:

1. Key Takeaways
Provide a short summary of the overall feedback in 3-5 bullet points. Focus only on the main issues or areas of satisfaction.
Include both positive and negative themes, but prioritize the most important and impactful points.
Limit each point to 1-2 sentences.
Before finalizing each point, take a moment to reflect on why each issue might be present (e.g., systemic problems, temporary issues, resource constraints, etc.)

2. Suggested Improvements
Based on the overall feedback, propose 2-3 high-level, actionable measures that the company could take to address the most pressing issues and enhance overall performance or satisfaction.
Each suggestion should be brief, directly tied to the feedback, and strategic in nature.
Think about short-term vs long-term solutions and consider the feasibility of each suggestion.

3. Output
Present your findings in a structured way with clear section headings, bullet points for easy scanning, and a consise, direct and professional tone suitable for leadership review.

This is the employee feedback data: {summary_text}
""")

# 2. Class for structured output
class ReportForLeadership(BaseModel):
    "Raport for the leadership of an IT-company on results of an internal IT-survey."
    snappy_title : str = Field(
        description="A fitting title for the report. Must begin with '# ' to ensure easy markdown formatting."
    )
    intro : str = Field(
        description="1 sentence describing thepurpose of the report." 
    )
    key_takeaways : str = Field(
        description="3-5 bulletpoints describing the key-takeaways. Limit each point to 1-2 sentences." 
    )
    suggested_improvements : str = Field(
        description="2-3 actionable measures for the company. Keep it brief."
    )
    outro: str = Field(
        description="1 sentence ending for the report. Be creative." 
    )

# 3. Report-chain
report_chain_structured_output = report_prompt | llm.with_structured_output(
    ReportForLeadership,
    method="json_schema",
    strict=True,
)

# 4. Kall modellen med oppsummeringen av kategoriene
report = report_chain_structured_output.invoke({"summary_text": summary.summary})



: 

: 

In [ ]:
report

: 

: 

In [ ]:
display(Markdown(
    "\n\n".join([report.snappy_title,
                 report.intro,
                 '## Key-takeaways',report.key_takeaways,
                 '## Suggested improvements',report.suggested_improvements,
                 report.outro])
                 
))

: 

: 

## Fantastisk! 
Du har nå fullført kurset *LLM for alle*, og jobbet deg gjennom hvordan du kan bruke metoder og verktøy som LCEL og structured output til å....
For de aller ivrigste har vi laget en ekstraoppgave helt til slutt - prøv deg frem hvis du har tid!

# Ekstraoppgave: Hvordan kunne vi gjort dette bedre?

Anta at du har fått levert resultatene fra denne spørreundersøkelsen i fanget av en stressa mellomleder som ber deg levere en rapport han kan presentere ledelsen. 
Gitt verktøyene du har fått en innføring i gjennom dette kursene (og kanskje andre erfaringer?), hvordan ville du løst oppgaven?

Ser du for eksempel noe som kunne vært forbedret i
- Rekkefølgen på måten vi leter etter kategorier?
- Legger vi får mye/lite vekt på inputen vi fikk om hva ledelsen "tror" kategoriene kommer til å være?
- Promptingen?
- Variablene eller type hintingen i pydantic-klassene?

Ville du kanskje gjort det helt annerledes? 
Now's your chance to try!

In [ ]:
# Prøv deg frem :))

# Hva er risikoene ved bruk av LLM til denne typen analyse?
Ved bruk av store språkmodeller (LLM) til analyse av åpne tekstsvar i spørreundersøkelser er risikoen generelt lavere enn i mer kritiske anvendelser som faktagenerering eller beslutningsstøtte. Likevel finnes det enkelte utfordringer som bør tas på alvor for å sikre kvalitet og transparens i analysen.
 
Skjevheter og bias
LLM-er kan tolke og vekte visse formuleringer eller temaer ulikt basert på mønstre fra treningsdataene. Dette kan føre til at noen svar over- eller underrepresenteres i visse kategorier avhengig av hvordan modellen "forstår" teksten. For eksempel kan emosjonelt ladede eller språklig avanserte svar få uforholdsmessig stor plass, mens mer nøkterne eller utypiske svar blir feilklassifisert. Særlig ved tolkning av følsomme temaer er det viktig å være oppmerksom på disse tendensene.
 
Hallusinasjon – mindre relevant, men ikke fraværende
I denne typen oppgave, hvor modellen hovedsakelig skal kategorisere eller oppsummere eksisterende tekstsvar er risikoen for såkalte hallusinasjoner (altså at modellen «finner opp» informasjon) mindre, men ikke helt ubetydelig. Det kan forekomme at modellen overfortolker et kort eller tvetydig svar og tillegger det en mening som ikke eksplisitt finnes i teksten. Dette kan føre til at enkelte svar havner i feil kategori eller påvirker tolkningen av sentiment eller tematikk.
 
Behov for kvalitetssikring
Selv med høy presisjon og effektivitet bør man ikke ta modellens resultater for gitt. Det anbefales å gjennomføre stikkprøver, sammenligne med manuell koding og evaluere nøyaktigheten på klassifiseringene. Det kan være veldig nyttig å kombinere automatisk kategorisering med menneskelig vurdering både i startfasen og underveis.